In [1]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from keras.optimizers import Adam


Using TensorFlow backend.


In [2]:
def build_discriminator():
    model = Sequential()
    model.add(Dense(41, input_dim=41, activation='relu'))  # discriminator takes 41 values from our dataset
    model.add(Dense(30, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # outputs 0 to 1, 1 being read and 0 being fake

    # model.summary()

    attack = Input(shape=(41,))
    validity = model(attack)

    return Model(attack, validity)

In [3]:
def build_generator(hidden1, hidden2, hidden3):
    model = Sequential()
    model.add(Dense(hidden1, input_dim=41))  # arbitrarily selected 100 for our input noise vector?
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(hidden2))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(hidden3))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(41, activation='relu'))  # outputs a generated vector of the same size as our data (41)

    # model.summary()

    noise = Input(shape=(41,))
    attack = model(noise)
    return Model(noise, attack)

In [4]:
def trainGAN(gen_hidden1, gen_hidden2, gen_hidden3):
    batch_size = 256
    epochs = 7000
    optimizer = Adam(0.0002, 0.5)
    
    dataframe = pd.read_csv('normalAndIpsweep.csv').sample(500) # sample 100 data points randomly from the csv
    
    # apply "le.fit_transform" to every column (usually only works on 1 column)
    le = LabelEncoder()
    dataframe_encoded = dataframe.apply(le.fit_transform)
    dataset = dataframe_encoded.values
    
    #to visually judge results
    print("Real ipsweep attacks:")
    print(dataset[:2])
    
    # Set X as our input data and Y as our label
    X_train = dataset[:, 0:41].astype(float)
    Y_train = dataset[:, 41]
    
    # labels for data. 1 for valid attacks, 0 for fake (generated) attacks
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    # build the discriminator portion
    discriminator = build_discriminator();
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # build the generator portion
    generator = build_generator(gen_hidden1, gen_hidden2, gen_hidden3)
    
    #input and output of our combined model
    z = Input(shape=(41,))
    attack = generator(z)
    validity = discriminator(attack)
    
    # build combined model from generator and discriminator
    combined = Model(z, validity)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    #break condition for training (when diverging)
    loss_increase_count = 0;
    prev_g_loss = 0;
    
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------
        
        # selecting batch_size random attacks from our training data
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        attacks = X_train[idx]
        
        # generate a matrix of noise vectors
        noise = np.random.normal(0, 1, (batch_size, 41))
        
        # create an array of generated attacks
        gen_attacks = generator.predict(noise)
        
        # loss functions, based on what metrics we specify at model compile time
        d_loss_real = discriminator.train_on_batch(attacks, valid)
        d_loss_fake = discriminator.train_on_batch(gen_attacks, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # generator loss function
        g_loss = combined.train_on_batch(noise, valid)
        
        if epoch % 100 == 0:
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [Loss change: %.3f, Loss increases: %.0f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss, g_loss - prev_g_loss, loss_increase_count))
        
        # if our generator loss icreased this iteration, increment the counter by 1
        if (g_loss - prev_g_loss) > 0:
            loss_increase_count = loss_increase_count + 1
        else: 
            loss_increase_count = 0  # otherwise, reset it to 0, we are still training effectively
            
        prev_g_loss = g_loss
            
        if loss_increase_count > 5:
            print('Stoping on iteration: ', epoch)
            break
            
        if epoch % 20 == 0:
            f = open("GANresultsIpsweep.txt", "a")
            np.savetxt("GANresultsIpsweep.txt", gen_attacks, fmt="%.0f")
            f.close()
            
    # peek at our results
    results = np.loadtxt("GANresultsIpsweep.txt")
    print("Generated ipsweep attacks: ")
    print(results[:2])
        
        


In [ ]:
# Initialize Random Number Generator
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load dataset

dataframe = pd.read_csv("normalAndIpsweep.csv")#, header=True) 

# samples 10000 random data points from 500k
dataframe = dataframe.sample(n=15000)
# LabelEncoder, turns all our categorical data into integers
le = LabelEncoder()

# apply "le.fit_transform" to every column (usually only works on 1 column)
dataframe_encoded = dataframe.apply(le.fit_transform)
attack_labels = le.classes_
indices_of_ipsweep = np.where(attack_labels == 'ipsweep.')
ipsweep_index = indices_of_ipsweep[0]
dataset = dataframe_encoded.values

print(attack_labels)
print(ipsweep_index)

#Set X as our input data and Y as our label
X = dataset[:,0:41].astype(float)
Y = dataset[:,41]


['ipsweep.' 'normal.']
[0]


In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# print(dummy_y)
#print(len(dummy_y[0]))
num_of_classes = len(dummy_y[0])  # the length of dummy y is the number of classes we have in our small sample
# since we are randomly sampling from a large dataset, we might not get 1 of every class in our sample
# we need to set output layer to be equal to the length of our dummy_y vectors
print(num_of_classes)


2


In [ ]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    
    inputs = 41
    hidden_layer1 = 10
    hidden_layer2 = 5
    hidden_layer3 = 0
    outputs = num_of_classes  #needs to be this variable in case we forget to sample. Could end up having 10 classes or 12, etc
    
    model.add(Dense(hidden_layer1, input_dim=inputs, activation='relu'))
    if hidden_layer2 != 0:
        model.add(Dense(hidden_layer2, activation='relu'))
    if hidden_layer3 != 0:
        model.add(Dense(hidden_layer3, activation='relu'))
    model.add(Dense(outputs, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #optimizer=adam
    return model

In [ ]:
#for i in range(0,10):
estimator = KerasClassifier(build_fn=baseline_model, epochs=32, batch_size=200, verbose=2)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
y_pred = cross_val_predict(estimator, X, dummy_y, cv=kfold)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)

trained_classifier = estimator.fit(X, Y)
print(type(estimator))

cm = confusion_matrix(Y, y_pred)
print(cm)
print("total: " + str(cm.sum()))
print("accuracy: " + str(np.trace(cm) / cm.sum()))
print("Matthews correlation coefficient: " + str(matthews_corrcoef(Y, y_pred)))



print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

f = open("discriminatorResults.txt", "a+")
f.write("TP: %d, FP: %d, FN: %d, TN: %d\n" % (cm[0][0], cm[0][1], cm[1][0], cm[1][1]))
f.close()


Epoch 1/32
 - 0s - loss: 2.5506 - acc: 0.6452
Epoch 2/32
 - 0s - loss: 0.2648 - acc: 0.8389
Epoch 3/32
 - 0s - loss: 0.0820 - acc: 0.9615
Epoch 4/32
 - 0s - loss: 0.0341 - acc: 0.9879
Epoch 5/32
 - 0s - loss: 0.0230 - acc: 0.9940
Epoch 6/32
 - 0s - loss: 0.0175 - acc: 0.9951
Epoch 7/32
 - 0s - loss: 0.0142 - acc: 0.9960
Epoch 8/32
 - 0s - loss: 0.0117 - acc: 0.9968
Epoch 9/32
 - 0s - loss: 0.0101 - acc: 0.9972
Epoch 10/32
 - 0s - loss: 0.0092 - acc: 0.9977
Epoch 11/32
 - 0s - loss: 0.0084 - acc: 0.9980
Epoch 12/32
 - 0s - loss: 0.0076 - acc: 0.9984
Epoch 13/32
 - 0s - loss: 0.0070 - acc: 0.9985
Epoch 14/32
 - 0s - loss: 0.0064 - acc: 0.9988
Epoch 15/32
 - 0s - loss: 0.0061 - acc: 0.9987
Epoch 16/32
 - 0s - loss: 0.0057 - acc: 0.9989
Epoch 17/32
 - 0s - loss: 0.0055 - acc: 0.9987
Epoch 18/32
 - 0s - loss: 0.0052 - acc: 0.9989
Epoch 19/32
 - 0s - loss: 0.0050 - acc: 0.9989
Epoch 20/32
 - 0s - loss: 0.0049 - acc: 0.9988
Epoch 21/32
 - 0s - loss: 0.0047 - acc: 0.9991
Epoch 22/32
 - 0s - lo

Epoch 17/32
 - 0s - loss: 0.0054 - acc: 0.9990
Epoch 18/32
 - 0s - loss: 0.0050 - acc: 0.9992
Epoch 19/32
 - 0s - loss: 0.0046 - acc: 0.9992
Epoch 20/32
 - 0s - loss: 0.0043 - acc: 0.9994
Epoch 21/32
 - 0s - loss: 0.0041 - acc: 0.9993
Epoch 22/32
 - 0s - loss: 0.0038 - acc: 0.9993
Epoch 23/32
 - 0s - loss: 0.0037 - acc: 0.9994
Epoch 24/32
 - 0s - loss: 0.0035 - acc: 0.9995
Epoch 25/32
 - 0s - loss: 0.0033 - acc: 0.9994
Epoch 26/32
 - 0s - loss: 0.0031 - acc: 0.9995
Epoch 27/32
 - 0s - loss: 0.0029 - acc: 0.9996
Epoch 28/32
 - 0s - loss: 0.0029 - acc: 0.9994
Epoch 29/32
 - 0s - loss: 0.0027 - acc: 0.9997
Epoch 30/32
 - 0s - loss: 0.0026 - acc: 0.9996
Epoch 31/32
 - 0s - loss: 0.0026 - acc: 0.9996
Epoch 32/32
 - 0s - loss: 0.0025 - acc: 0.9998
Epoch 1/32
 - 0s - loss: 1.5862 - acc: 0.7381
Epoch 2/32
 - 0s - loss: 0.0930 - acc: 0.9672
Epoch 3/32
 - 0s - loss: 0.0454 - acc: 0.9836
Epoch 4/32
 - 0s - loss: 0.0349 - acc: 0.9869
Epoch 5/32
 - 0s - loss: 0.0283 - acc: 0.9894
Epoch 6/32
 - 0s -

Epoch 1/32
 - 0s - loss: 3.6649 - acc: 0.6054
Epoch 2/32
 - 0s - loss: 0.2914 - acc: 0.8651
Epoch 3/32
 - 0s - loss: 0.0719 - acc: 0.9860
Epoch 4/32
 - 0s - loss: 0.0486 - acc: 0.9890
Epoch 5/32
 - 0s - loss: 0.0401 - acc: 0.9932
Epoch 6/32
 - 0s - loss: 0.0350 - acc: 0.9947
Epoch 7/32
 - 0s - loss: 0.0315 - acc: 0.9959
Epoch 8/32
 - 0s - loss: 0.0269 - acc: 0.9964
Epoch 9/32
 - 0s - loss: 0.0167 - acc: 0.9969
Epoch 10/32
 - 0s - loss: 0.0145 - acc: 0.9973
Epoch 11/32
 - 0s - loss: 0.0132 - acc: 0.9979
Epoch 12/32
 - 0s - loss: 0.0120 - acc: 0.9981
Epoch 13/32
 - 0s - loss: 0.0112 - acc: 0.9981
Epoch 14/32
 - 0s - loss: 0.0106 - acc: 0.9982
Epoch 15/32
 - 0s - loss: 0.0100 - acc: 0.9983
Epoch 16/32
 - 0s - loss: 0.0094 - acc: 0.9985
Epoch 17/32
 - 0s - loss: 0.0089 - acc: 0.9986
Epoch 18/32
 - 0s - loss: 0.0084 - acc: 0.9986
Epoch 19/32
 - 0s - loss: 0.0080 - acc: 0.9986
Epoch 20/32
 - 0s - loss: 0.0079 - acc: 0.9987
Epoch 21/32
 - 0s - loss: 0.0078 - acc: 0.9987
Epoch 22/32
 - 0s - lo

Epoch 17/32
 - 0s - loss: 0.0053 - acc: 0.9985
Epoch 18/32
 - 0s - loss: 0.0048 - acc: 0.9987
Epoch 19/32
 - 0s - loss: 0.0048 - acc: 0.9986
Epoch 20/32
 - 0s - loss: 0.0046 - acc: 0.9986
Epoch 21/32
 - 0s - loss: 0.0045 - acc: 0.9987
Epoch 22/32
 - 0s - loss: 0.0043 - acc: 0.9988
Epoch 23/32
 - 0s - loss: 0.0042 - acc: 0.9988
Epoch 24/32
 - 0s - loss: 0.0041 - acc: 0.9987
Epoch 25/32
 - 0s - loss: 0.0036 - acc: 0.9989
Epoch 26/32
 - 0s - loss: 0.0036 - acc: 0.9988
Epoch 27/32
 - 0s - loss: 0.0036 - acc: 0.9989
Epoch 28/32
 - 0s - loss: 0.0035 - acc: 0.9989
Epoch 29/32
 - 0s - loss: 0.0035 - acc: 0.9987
Epoch 30/32
 - 0s - loss: 0.0038 - acc: 0.9988
Epoch 31/32
 - 0s - loss: 0.0031 - acc: 0.9990
Epoch 32/32
 - 0s - loss: 0.0030 - acc: 0.9989
Epoch 1/32
 - 1s - loss: 5.3241 - acc: 0.6633
Epoch 2/32
 - 0s - loss: 2.3154 - acc: 0.8377
Epoch 3/32
 - 0s - loss: 0.1452 - acc: 0.9799
Epoch 4/32
 - 0s - loss: 0.0300 - acc: 0.9938
Epoch 5/32
 - 0s - loss: 0.0197 - acc: 0.9950
Epoch 6/32
 - 0s -

In [ ]:
"""
f = open("GeneratorHypersAbove50percentAccuracy.txt", "w")
f.write("""""" Hidden layer counts for Generator model that resulted in over 50% generated attacks labeled correctly:
    ------------------------------------------------------------------------------------------------
    """""")
f.close()
"""

while(1):
    # generate random numbers for the hidden layer sizes of our generator
    gen_hidden1 = 30 # np.random.randint(1, 101)
    gen_hidden2 = 49 # np.random.randint(1, 101)
    gen_hidden3 = 62 # np.random.randint(1, 101)
    
    i = 0
    
    
    # train 5 times on each setup, in case we get unlucky initalization on an otherwise good setup
    while i < 100:
        # create a unique filename in case we want to store the results (good accuracy)
        result_filename = "GANresultsIpsweep%.0f%.0f%.0fiter%.0ftry2.txt" % (gen_hidden1, gen_hidden2, gen_hidden3, i)

        trainGAN(gen_hidden1, gen_hidden2, gen_hidden3)
        
        # load generate attacks from file
        results = np.loadtxt("GANresultsIpsweep.txt")

        # predict attack lables (as encoded integers)
        y_pred = estimator.predict(results)
        print(y_pred)

        # create appropriate labels for our generated ipsweep attacks
        ipsweep_labels = np.full((len(results),), ipsweep_index[0])

        # convert integer labels back to string, get all unique strings and their count
        predicted_as_label = attack_labels[y_pred]
        unique_labels = np.unique(predicted_as_label)

        for label in unique_labels:
            print("Attack type: %s     number predicted:  %.0f" % (label, len(np.where(predicted_as_label == label)[0])))
    
        print()
        # create a confusion matrix of the results
        cm = confusion_matrix(ipsweep_labels, y_pred)
        
        accuracy = np.trace(cm) / cm.sum()
        print(cm)
        print("total: " + str(cm.sum()))
        print("accuracy: " + str(accuracy))
        
        if accuracy > .50:
            f = open("GeneratorHypersAbove50percentAccuracyIpsweep.txt", "a")
            f.write("""
            
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accuracy: %.3f
Generator hidden layer 1 size: %.0f
Generator hidden layer 2 size: %.0f
Generator hidden layer 3 size: %.0f
Iteration %.0f
Result file name: %s
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~""" % (accuracy, gen_hidden1, gen_hidden2, gen_hidden3, i, result_filename))
            f.close()
            
            f = open(result_filename, "w")
            f.close()
            np.savetxt(result_filename, results, fmt="%.0f")
        
        i = i + 1
            


Real ipsweep attacks:
[[  0   0   3   2   5   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   1   0   0   0 106  52   0  32
   30   0   0   0   0   0]
 [  0   1   7   2  91  96   0   0   0   0   0   1   0   0   0   0   0   0
    0   0   0   0  20  20   0   0   0   0   1   0   0  61 118  52   0   1
    2   0   0   0   0   1]]
0 [D loss: 4.243306, acc.: 55.47%] [G loss: 0.886255] [Loss change: 0.886, Loss increases: 0]
100 [D loss: 0.917396, acc.: 50.00%] [G loss: 0.213865] [Loss change: 0.009, Loss increases: 0]
200 [D loss: 1.240550, acc.: 50.20%] [G loss: 0.140731] [Loss change: 0.007, Loss increases: 2]
300 [D loss: 1.531011, acc.: 50.00%] [G loss: 0.086049] [Loss change: 0.003, Loss increases: 0]
400 [D loss: 1.601152, acc.: 49.41%] [G loss: 0.076424] [Loss change: 0.008, Loss increases: 1]
500 [D loss: 1.603209, acc.: 50.00%] [G loss: 0.073278] [Loss change: 0.015, Loss increases: 0]
600 [D loss: 1.647684, acc.: 49.80%] [G loss: 0.05848

0 [D loss: 0.395743, acc.: 50.20%] [G loss: 0.549041] [Loss change: 0.549, Loss increases: 0]
100 [D loss: 0.686197, acc.: 50.00%] [G loss: 0.341786] [Loss change: 0.004, Loss increases: 0]
200 [D loss: 0.736010, acc.: 50.00%] [G loss: 0.304559] [Loss change: -0.009, Loss increases: 1]
300 [D loss: 0.737257, acc.: 50.00%] [G loss: 0.302923] [Loss change: 0.006, Loss increases: 0]
400 [D loss: 0.738517, acc.: 50.20%] [G loss: 0.294411] [Loss change: 0.005, Loss increases: 0]
500 [D loss: 0.737366, acc.: 50.00%] [G loss: 0.294579] [Loss change: -0.005, Loss increases: 1]
600 [D loss: 0.704277, acc.: 50.00%] [G loss: 0.301293] [Loss change: -0.005, Loss increases: 3]
700 [D loss: 0.700958, acc.: 50.00%] [G loss: 0.310164] [Loss change: 0.003, Loss increases: 2]
800 [D loss: 0.672724, acc.: 50.00%] [G loss: 0.319673] [Loss change: 0.006, Loss increases: 0]
900 [D loss: 0.698695, acc.: 50.20%] [G loss: 0.318968] [Loss change: -0.002, Loss increases: 1]
1000 [D loss: 0.686722, acc.: 50.20%] 

0 [D loss: 3.209314, acc.: 46.29%] [G loss: 0.774678] [Loss change: 0.775, Loss increases: 0]
100 [D loss: 1.049771, acc.: 50.00%] [G loss: 0.187901] [Loss change: -0.000, Loss increases: 0]
200 [D loss: 1.379289, acc.: 50.00%] [G loss: 0.102037] [Loss change: -0.011, Loss increases: 1]
300 [D loss: 1.471781, acc.: 50.00%] [G loss: 0.084925] [Loss change: -0.006, Loss increases: 1]
400 [D loss: 1.467327, acc.: 50.00%] [G loss: 0.074631] [Loss change: -0.007, Loss increases: 1]
500 [D loss: 1.515850, acc.: 50.00%] [G loss: 0.073300] [Loss change: -0.003, Loss increases: 1]
600 [D loss: 1.459590, acc.: 50.00%] [G loss: 0.072265] [Loss change: -0.001, Loss increases: 1]
700 [D loss: 1.440288, acc.: 50.00%] [G loss: 0.081986] [Loss change: 0.005, Loss increases: 1]
800 [D loss: 1.434589, acc.: 50.00%] [G loss: 0.075411] [Loss change: 0.001, Loss increases: 0]
900 [D loss: 1.436323, acc.: 50.00%] [G loss: 0.070522] [Loss change: -0.006, Loss increases: 1]
1000 [D loss: 1.364991, acc.: 50.00

4200 [D loss: 0.905249, acc.: 49.02%] [G loss: 0.265193] [Loss change: -0.002, Loss increases: 1]
4300 [D loss: 0.921577, acc.: 49.41%] [G loss: 0.265123] [Loss change: -0.008, Loss increases: 4]
4400 [D loss: 0.885304, acc.: 49.22%] [G loss: 0.289322] [Loss change: 0.013, Loss increases: 3]
4500 [D loss: 0.903182, acc.: 50.00%] [G loss: 0.279099] [Loss change: 0.006, Loss increases: 0]
4600 [D loss: 0.880373, acc.: 49.41%] [G loss: 0.288753] [Loss change: 0.004, Loss increases: 1]
4700 [D loss: 0.917434, acc.: 50.00%] [G loss: 0.290057] [Loss change: -0.021, Loss increases: 4]
4800 [D loss: 0.915948, acc.: 50.78%] [G loss: 0.301615] [Loss change: -0.006, Loss increases: 1]
4900 [D loss: 0.916690, acc.: 50.20%] [G loss: 0.306743] [Loss change: -0.011, Loss increases: 0]
5000 [D loss: 0.920933, acc.: 50.20%] [G loss: 0.307357] [Loss change: -0.001, Loss increases: 0]
5100 [D loss: 0.919472, acc.: 49.80%] [G loss: 0.314417] [Loss change: -0.022, Loss increases: 3]
5200 [D loss: 1.019882,

1800 [D loss: 0.685081, acc.: 50.00%] [G loss: 0.334450] [Loss change: -0.008, Loss increases: 3]
1900 [D loss: 0.660968, acc.: 49.41%] [G loss: 0.347729] [Loss change: 0.001, Loss increases: 1]
2000 [D loss: 0.651870, acc.: 50.39%] [G loss: 0.350491] [Loss change: 0.005, Loss increases: 1]
2100 [D loss: 0.640406, acc.: 50.59%] [G loss: 0.353333] [Loss change: -0.011, Loss increases: 1]
2200 [D loss: 0.659021, acc.: 50.39%] [G loss: 0.366846] [Loss change: -0.007, Loss increases: 3]
2300 [D loss: 0.653670, acc.: 50.00%] [G loss: 0.367367] [Loss change: -0.005, Loss increases: 0]
2400 [D loss: 0.623344, acc.: 51.17%] [G loss: 0.391585] [Loss change: 0.001, Loss increases: 2]
2500 [D loss: 0.623824, acc.: 50.78%] [G loss: 0.383047] [Loss change: -0.012, Loss increases: 1]
2600 [D loss: 0.593148, acc.: 50.98%] [G loss: 0.397119] [Loss change: 0.002, Loss increases: 0]
2700 [D loss: 0.579292, acc.: 51.76%] [G loss: 0.413155] [Loss change: 0.005, Loss increases: 1]
2800 [D loss: 0.582215, a

1700 [D loss: 1.359844, acc.: 50.00%] [G loss: 0.076024] [Loss change: 0.002, Loss increases: 0]
1800 [D loss: 1.336883, acc.: 50.00%] [G loss: 0.076473] [Loss change: -0.000, Loss increases: 1]
1900 [D loss: 1.325411, acc.: 50.00%] [G loss: 0.079597] [Loss change: -0.005, Loss increases: 1]
2000 [D loss: 1.296787, acc.: 50.00%] [G loss: 0.083391] [Loss change: -0.001, Loss increases: 0]
Stoping on iteration:  2097
Generated ipsweep attacks: 
[[ 0.  0.  5.  0. 19. 15.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  5. 20. 17.  3.  0.  5.
   6.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  9. 36. 11.  0.  0.
   0.  0.  0.  0.  0.]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

0 [D loss: 4.983637, acc.: 59.18%] [G loss: 0.881952] [Loss change: 0.882, Loss increases: 0]
100 [D loss: 1.058786, acc.: 50.00%] [G loss: 0.174124] [Loss change: 0.004, Loss increases: 0]
200 [D loss: 1.517516, acc.: 50.00%] [G loss: 0.087038] [Loss change: 0.010, Loss increases: 0]
300 [D loss: 1.629999, acc.: 50.00%] [G loss: 0.054697] [Loss change: -0.006, Loss increases: 1]
400 [D loss: 1.687740, acc.: 50.00%] [G loss: 0.050389] [Loss change: -0.010, Loss increases: 1]
500 [D loss: 1.651968, acc.: 49.80%] [G loss: 0.051065] [Loss change: 0.001, Loss increases: 1]
600 [D loss: 1.672471, acc.: 50.00%] [G loss: 0.051611] [Loss change: -0.000, Loss increases: 1]
700 [D loss: 1.618542, acc.: 50.00%] [G loss: 0.054708] [Loss change: 0.002, Loss increases: 0]
800 [D loss: 1.646634, acc.: 50.00%] [G loss: 0.053793] [Loss change: 0.005, Loss increases: 0]
900 [D loss: 1.665639, acc.: 50.00%] [G loss: 0.052563] [Loss change: 0.006, Loss increases: 0]
Stoping on iteration:  916
Generated ip

2800 [D loss: 0.834570, acc.: 47.66%] [G loss: 0.331299] [Loss change: 0.002, Loss increases: 0]
2900 [D loss: 0.764748, acc.: 49.22%] [G loss: 0.333233] [Loss change: 0.002, Loss increases: 1]
3000 [D loss: 0.800235, acc.: 49.02%] [G loss: 0.317120] [Loss change: -0.008, Loss increases: 0]
3100 [D loss: 0.819245, acc.: 49.22%] [G loss: 0.319069] [Loss change: 0.012, Loss increases: 0]
3200 [D loss: 0.793092, acc.: 48.44%] [G loss: 0.333011] [Loss change: 0.005, Loss increases: 0]
3300 [D loss: 0.786159, acc.: 48.24%] [G loss: 0.340068] [Loss change: 0.010, Loss increases: 0]
3400 [D loss: 0.758458, acc.: 48.44%] [G loss: 0.348226] [Loss change: -0.006, Loss increases: 1]
3500 [D loss: 0.764511, acc.: 48.83%] [G loss: 0.362882] [Loss change: -0.010, Loss increases: 1]
3600 [D loss: 0.723966, acc.: 49.80%] [G loss: 0.363958] [Loss change: -0.003, Loss increases: 0]
3700 [D loss: 0.766013, acc.: 50.20%] [G loss: 0.363319] [Loss change: -0.016, Loss increases: 0]
3800 [D loss: 0.693944, a